In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

****1-Importing the Data****

In [ ]:
df_train=pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")
df_test = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/test.csv.zip")

**Exploring the Data**

In [ ]:
df_train.head(10)

Consists of several Cities 

In [ ]:
df_test.head()

In [ ]:
print("Train Dim:",df_train.shape)
print("Test Dim:",df_test.shape)

Lets Test for null values


In [ ]:
df_train.info()

No Null Values Found, 9 Float values including Revenue 




Lets look at the description

In [ ]:
df_train.describe(include=['object'])

In [ ]:
df_train['City'].unique()
df_train['City Group'].unique()
df_train['Type'].unique()


In [ ]:
df_train.describe()

****2 - Exploratory Data Analysis****

Let us perform EDA by finding out:
*        What is the best City to open a restraunt 
*        What is the Best type of restraunt
*        What is the best time to open a restraunt

In [ ]:
df_train['revenue'].describe()

*  As the mean is > 50% mark must be positively skewed 
*  Also the 75% mark and the Max value suggests the presence of outliers

In [ ]:
f,ax = plt.subplots(1,2)
sns.distplot(df_train['revenue'],ax=ax[0])
sns.boxplot(df_train['revenue'])
plt.tight_layout()

* As observed from the Describe list the data is positively Skewed
* Removing the outliers which are>1e7

In [ ]:
df_train= df_train[df_train['revenue']<1e7]
df_train.reset_index(drop=True).head()

In [ ]:

print("Big Cities:",df_train.loc[df_train["City Group"]=="Big Cities"]["City"].unique())
print("No of Restraunts in Big Cities:",df_train.loc[df_train["City Group"]=="Big Cities"].shape[0])

Istanbul, Anakara and Izmir Are bigger cities 


In [ ]:
sns.catplot(x='City', y='revenue', 
            data=df_train,
            kind='box',
            height=20, aspect=1.5)

In [ ]:
df_train.City.value_counts()

Inference:
* Cities like Afyonkarahisar,Edirne,Elazığ might be performing well but the data is insuffient to draw conslusions.  
* Although, Cities like Trabzon,Aydın,Muğla,Eskişehir show a consistant revenue, with lesser samples cannot get to a statistical Conclusion. 
* The Median of the Istanbul is relatively higher compared to the qualified Cities.
* **Answering the Question the best city is ISTANBUL**

In [ ]:
sns.catplot(x='Type', y='revenue', 
            data=df_train,
            kind='box',
            height=10, aspect=1.1)
plt.figure()
sns.distplot(df_train.loc[df_train['Type']=='IL']['revenue'])
sns.distplot(df_train.loc[df_train['Type']=='FC']['revenue'])

* The tightness of the plot infers consistancy in genertion of revenue
* ****From the above boxplot it can be inferred that large portion of samples for 'IL' type have higher revenue****
 

Lets us Check how are the Restraunt Types performing in Istanbul 

In [ ]:
sns.catplot(x='Type', y='revenue', 
            data=df_train.loc[df_train["City"]=="İstanbul"],
            kind='box',
            height=10, aspect=1.1)
plt.figure()
sns.distplot(df_train.loc[(df_train['Type']=='IL')&(df_train['City']=='İstanbul')]['revenue'])
sns.distplot(df_train.loc[(df_train['Type']=='FC')&(df_train['City']=='İstanbul')]['revenue'])


*  The distribution of 'IL' in Istanbul will be a normal distribution
*  **The performance in terms of revenue in 'Istanbul' is better for FC Type**
 

In [ ]:
df_train.head()

In [ ]:
df_train['Open Date'] = pd.to_datetime(df_train['Open Date'])
df_test['Open Date'] = pd.to_datetime(df_test['Open Date'])

In [ ]:
print("Analysis is in the time frame From:",min(df_train['Open Date']),"To:",max(df_train['Open Date']))
print("Prediction is in the time frame From:",min(df_test['Open Date']),"To:",max(df_test['Open Date']))

In [ ]:
df_train['Day']=df_train['Open Date'].dt.day

In [ ]:
 df_train['Month']=df_train['Open Date'].dt.month

In [ ]:
df_train['Year']=df_train['Open Date'].dt.year

In [ ]:
def get_pie(City,Type):
    df_train_Q1 = df_train.loc[(df_train["Month"]<=3) & (df_train["City"]==City) & (df_train["Type"]==Type)]
    df_train_Q2 = df_train.loc[(df_train["Month"]>=4) & (df_train["Month"]<=6)& (df_train["City"]==City)& (df_train["Type"]==Type)]
    df_train_Q3 = df_train.loc[(df_train["Month"]>=7) & (df_train["Month"]<=9) & (df_train["City"]==City)& (df_train["Type"]==Type)]
    df_train_Q4 = df_train.loc[(df_train["Month"]>=10) & (df_train["Month"]<=12) & (df_train["City"]==City)& (df_train["Type"]==Type)]

    labels = 'Q1', 'Q2', 'Q3', 'Q4'
    sizes = [df_train_Q1['revenue'].sum(), df_train_Q2['revenue'].sum(), df_train_Q3['revenue'].sum(), df_train_Q4['revenue'].sum()]
    colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
    explode = (0, 0, 0, 0)  # explode 4th slice

    # Plot
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
    autopct='%1.1f%%', shadow=True, startangle=140)

    plt.axis('equal')
    plt.show()

    

In [ ]:
get_pie('İstanbul','FC')


**The best time to open a FC in Istanbul is in 4th Quarter of the year(U.S)**

****2- Exploratory Data Analysis****

In [ ]:
df_train=df_train.drop(['Id'],axis=1)

In [ ]:
df_test['Day']=df_test['Open Date'].dt.day
df_test['Month']=df_test['Open Date'].dt.month
df_test['Year']=df_test['Open Date'].dt.year


In [ ]:
#df_train['Active Days']= (max(df_train['Open Date'])-(df_train['Open Date'])).astype('timedelta64[D]').astype(int)
#df_train['Open Date']= df_train['Active Days']
#df_train.rename(columns={'Open Date':'Days Active'},inplace=True)
#df_train=df_train.drop(['Active Days'],axis=1)
#df_train.head()

In [ ]:
#df_test['Active Days']=(max(df_test['Open Date'])-(df_test['Open Date'])).astype('timedelta64[D]').astype(int)
#df_test['Open Date']= df_test['Active Days']
#df_test.rename(columns={'Open Date':'Days Active'},inplace=True)
#df_test=df_test.drop(['Active Days'],axis=1)
#df_test.head()


In [ ]:
num=df_train.select_dtypes([np.number]).columns.tolist()
df_train[num].head()

In [ ]:
plt.figure(figsize=(45,25))
sns.heatmap(df_train[num].corr(),annot=True)


Features P5,P8,P13,P15,P26,P37 have little Correlation 

Dropping them to avoid penality

In [ ]:
X_train= df_train.drop(['P5','P8','P13','P15','P26','P37','revenue'],axis=1)
X_train.head(X_train.shape[1])
y_train =df_train['revenue']
y_train.head(10)


In [ ]:
X_test = df_test.drop(['Id','P5','P8','P13','P15','P26','P37'],axis=1)
X_test.head()

In [ ]:
X_train.head()

In [ ]:
X_test.info()

****3-Regression****

****Data Preprocessing****

Concatinating X_train and X_test for One Hot Encoding 


In [ ]:
X=pd.concat([X_train,X_test],axis=0)
X.shape

In [ ]:
X

In [ ]:
X=X.drop(['Open Date'],axis=1)

In [ ]:
X=pd.get_dummies(X,columns=['City','City Group','Type','Day','Month','Year'],drop_first=True)

Seperating X_train and X_test

In [ ]:
X_train = X.iloc[:134,:]
X_train.tail()

In [ ]:
X_test=X.iloc[134:,:]
X_test.head()

Splitting the data into Training and Validation Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)


* **RMSE Observed :   1281693,N_est=500,RS=47**
* Fitting the Training Data to Random Forest Method


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics 


In [ ]:
regressor = RandomForestRegressor(n_estimators = 500,random_state=47,max_depth =50)
regressor.fit(X_train, y_train)
y_pred_val  = regressor.predict(X_val)
print('RMSE score through Random Forest : ',np.sqrt(metrics.mean_squared_error(y_val,y_pred_val)))
print('Score through Random Forest : ',regressor.score(X_train, y_train))

In [ ]:
y_pred = regressor.predict(X_test)

***XGBoosting***

In [ ]:
from xgboost import XGBRegressor
Regressor = XGBRegressor(n_estimators=500,random_state=47)
Regressor.fit(X_train, y_train)
# Predicting the Test set results
y_pred_val = Regressor.predict(X_val)
print('RMSE score through XGBoosting : ',np.sqrt(metrics.mean_squared_error(y_val,y_pred_val)))
print('Score through XGB : ',Regressor.score(X_train, y_train))

In [ ]:
y_pred = Regressor.predict(X_test)

In [ ]:
submission = pd.DataFrame({
        "Id": df_test["Id"],
        "Prediction": y_pred
    })
submission.to_csv('Restraunt_Revenue_RF.csv',header=True, index=False)